# S&P 500 Top 50 — Stock Probability Analyzer
**finance2** | Technical analysis notebook

This notebook uses the `lib/` package to:
1. Fetch OHLCV data for the top 50 S&P 500 stocks (via `yfinance`)
2. Compute 7 technical indicators per stock
3. Score each indicator on a 0–1 scale and blend into a composite probability
4. Visualise results with ranked charts and a heatmap

> ⚠️ **Disclaimer:** For educational purposes only. Not financial advice.

## 1 · Setup

In [ ]:
# Install dependencies if needed
import subprocess, sys

for pkg in ["yfinance", "matplotlib"]:
    try:
        __import__(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q"])

print("✓ Dependencies ready")

In [ ]:
import sys, warnings
warnings.filterwarnings("ignore")

# Add the repo root to path so 'lib' is importable
sys.path.insert(0, "..")

from lib import data, indicators, scoring, display
from datetime import datetime

print(f"✓ lib package loaded  |  Run date: {datetime.now():%Y-%m-%d %H:%M}")

## 2 · Configuration
Edit the cells below to customise the analysis.

In [ ]:
# ── Stock universe ──────────────────────────────────────────
# Use the default top-50 list, or supply your own tickers:
TICKERS = data.get_top_n_sp500(n=50, verbose=True)   # live top-50 by market cap (~90 s)
# TICKERS = data.TOP_50_SP500                        # hardcoded fallback (instant, offline)
# TICKERS = ["AAPL", "MSFT", "NVDA"]                 # custom subset for quick testing

# ── Indicator weights (must sum to 1.0) ──────────────────────
from lib.scoring import WEIGHTS
print("Current weights:")
for k, v in WEIGHTS.items():
    bar = "█" * round(v * 40)
    print(f"  {k:<12} {v:.2f}  {bar}")

## 3 · Fetch Market Data
Downloads approximately 400 days of adjusted OHLCV history for each ticker.

In [ ]:
print(f"Fetching data for {len(TICKERS)} stocks…\n")
ohlcv_data = data.fetch_all(TICKERS, lookback_days=400, delay=0.3, verbose=True)
print(f"\n✓ {len(ohlcv_data)} / {len(TICKERS)} stocks fetched successfully")

## 4 · Run Technical Analysis
Scores each stock across all 7 indicators and blends them into a composite probability.

In [ ]:
print("Scoring stocks…\n")
results = scoring.analyze_universe(ohlcv_data, verbose=True)
print(f"\n✓ Analysis complete  |  {len(results)} stocks scored")

## 5 · Summary Statistics

In [ ]:
import numpy as np

scores = [r["score"] for r in results]
print(f"  Stocks analysed : {len(results)}")
print(f"  Mean score      : {np.mean(scores):.4f}")
print(f"  Median score    : {np.median(scores):.4f}")
print(f"  Highest score   : {max(scores):.4f}  ({results[0]['ticker']})")
print(f"  Lowest score    : {min(scores):.4f}  ({results[-1]['ticker']})")

display.print_signal_distribution(results)

## 6 · Full Ranked Table

In [ ]:
display.print_summary_table(results)

## 7 · Score Chart
Horizontal bar chart — green = bullish signal, red = bearish.

In [ ]:
fig = display.plot_scores(results, top_n=50)
fig.savefig("score_chart.png", dpi=150, bbox_inches="tight")
fig

## 8 · Signal Distribution

In [ ]:
fig = display.plot_signal_distribution(results)
fig.savefig("signal_distribution.png", dpi=150, bbox_inches="tight")
fig

## 9 · Indicator Heatmap
Per-indicator scores for the top 20 stocks. Red = bearish, green = bullish.

In [ ]:
fig = display.plot_component_heatmap(results, top_n=20)
fig.savefig("indicator_heatmap.png", dpi=150, bbox_inches="tight")
fig

## 10 · Detailed Breakdown — Top 10

In [ ]:
display.print_detailed(results, top_n=10)

## 11 · Export Results

In [ ]:
df = display.results_to_dataframe(results)
print(f"DataFrame shape: {df.shape}")
df.head(10)

In [ ]:
display.save_csv(results, path="sp500_analysis.csv")
print("Done! Files saved:")
print("  sp500_analysis.csv")
print("  score_chart.png")
print("  signal_distribution.png")
print("  indicator_heatmap.png")

## 12 · Custom Single-Stock Deep Dive
Change `TICKER` to inspect any stock in detail.

In [ ]:
TICKER = "AAPL"

if TICKER in ohlcv_data:
    result = scoring.analyze(TICKER, ohlcv_data[TICKER])
    if result:
        print(f"\n{'═'*55}")
        print(f"  {result['ticker']}  —  ${result['price']:.2f}")
        print(f"  Composite score : {result['score']:.4f}  [{result['signal']}]")
        print(f"{'═'*55}")
        print(f"\n  Indicator values")
        print(f"  {'─'*40}")
        fields = [
            ("RSI",            f"{result['rsi']:.1f}",     f"score={result['score_rsi']:.3f}"),
            ("MACD",           f"{result['macd']:.4f}",    f"score={result['score_macd']:.3f}"),
            ("Bollinger %B",   f"{result['bb_pct_b']:.3f}",f"score={result['score_boll']:.3f}"),
            ("SMA 20/50/200",  f"{result['sma20']:.0f}/{result['sma50']:.0f}/{result['sma200']:.0f}",
                               f"score={result['score_ma']:.3f}"),
            ("Stochastic %K",  f"{result['stoch_k']:.1f}", f"score={result['score_stoch']:.3f}"),
            ("Momentum 5d/20d",f"{result['mom5']:+.2f}%/{result['mom20']:+.2f}%",
                               f"score={result['score_mom']:.3f}"),
            ("Volume trend",   f"{result['vol_trend']:.2f}×",f"score={result['score_vol']:.3f}"),
        ]
        for name, value, score_str in fields:
            print(f"  {name:<22} {value:<14}  {score_str}")
        print()
        flags = []
        if result['golden_cross']: flags.append("Golden Cross ✓")
        if result['above_sma200']: flags.append("Above SMA200 ✓")
        if flags:
            print("  Flags:", ", ".join(flags))
else:
    print(f"{TICKER} not found in fetched data.")